# Saving SCF results on disk and SCF checkpoints

For longer DFT calculations it is pretty standard to run them on a cluster
in advance and to perform postprocessing (band structure calculation,
plotting of density, etc.) at a later point and potentially on a different
machine.

To support such workflows DFTK offers the two functions `save_scfres`
and `load_scfres`, which allow to save the data structure returned
by `self_consistent_field` on disk or retrieve it back into memory,
respectively. For this purpose DFTK uses the
[JLD2.jl](https://github.com/JuliaIO/JLD2.jl) file format and Julia package.

> **Availability of `load_scfres`, `save_scfres` and checkpointing**
>
> As JLD2 is an optional dependency of DFTK these three functions are only
> available once one has *both* imported DFTK and JLD2 (`using DFTK`
> and `using JLD2`).

To illustrate the use of the functions in practice we will compute
the total energy of the O₂ molecule at PBE level. To get the triplet
ground state we use a collinear spin polarisation
(see Collinear spin and magnetic systems for details)
and a bit of temperature to ease convergence:

In [1]:
using DFTK
using LinearAlgebra
using JLD2
using PseudoPotentialData

d = 2.079  # oxygen-oxygen bondlength
a = 9.0    # size of the simulation box
lattice = a * I(3)
pseudopotentials = PseudoFamily("cp2k.nc.sr.pbe.v0_1.semicore.gth")
O = ElementPsp(:O, pseudopotentials)
atoms     = [O, O]
positions = d / 2a * [[0, 0, 1], [0, 0, -1]]
magnetic_moments = [1., 1.]

Ecut  = 10  # Far too small to be converged
model = model_DFT(lattice, atoms, positions;
                  functionals=PBE(),
                  temperature=0.02, smearing=Smearing.Gaussian(),
                  magnetic_moments)
basis = PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])

scfres = self_consistent_field(basis, tol=1e-2, ρ=guess_density(basis, magnetic_moments))
save_scfres("scfres.jld2", scfres);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   Diag   Δtime
---   ---------------   ---------   ---------   ------   ----   ------
  1   -27.64794311912                   -0.13    0.001    7.0    181ms
  2   -28.92464247803        0.11       -0.83    0.697    2.0    128ms
  3   -28.93124241356       -2.18       -1.13    1.204    2.5    109ms
  4   -28.93784087957       -2.18       -1.19    1.784    1.5   82.2ms
  5   -28.93913465916       -2.89       -1.00    2.000    2.0   89.9ms
  6   -28.93958509257       -3.35       -1.89    1.975    1.5   87.0ms
  7   -28.93961169594       -4.58       -3.04    1.985    1.0   79.9ms


In [2]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             16.7727913
    AtomicLocal         -58.4949975
    AtomicNonlocal      4.7085687 
    Ewald               -4.8994689
    PspCorrection       0.0044178 
    Hartree             19.3613047
    Xc                  -6.3913452
    Entropy             -0.0008827

    total               -28.939611695943

The `scfres.jld2` file could now be transferred to a different computer,
Where one could fire up a REPL to inspect the results of the above
calculation:

In [3]:
using DFTK
using JLD2
loaded = load_scfres("scfres.jld2")
propertynames(loaded)

(:α, :history_Δρ, :converged, :occupation, :occupation_threshold, :algorithm, :basis, :runtime_ns, :n_iter, :n_matvec, :history_Etot, :εF, :energies, :ρ, :timedout, :n_bands_converge, :eigenvalues, :ψ, :ham)

In [4]:
loaded.energies

Energy breakdown (in Ha):
    Kinetic             16.7727913
    AtomicLocal         -58.4949975
    AtomicNonlocal      4.7085687 
    Ewald               -4.8994689
    PspCorrection       0.0044178 
    Hartree             19.3613047
    Xc                  -6.3913452
    Entropy             -0.0008827

    total               -28.939611695943

Since the loaded data contains exactly the same data as the `scfres` returned by the
SCF calculation one could use it to plot a band structure, e.g.
`plot_bandstructure(load_scfres("scfres.jld2"))` directly from the stored data.

Notice that both `load_scfres` and `save_scfres` work by transferring all data
to/from the master process, which performs the IO operations without parallelisation.
Since this can become slow, both functions support optional arguments to speed up
the processing. An overview:
- `save_scfres("scfres.jld2", scfres; save_ψ=false)` avoids saving
  the Bloch wave, which is usually faster and saves storage space.
- `load_scfres("scfres.jld2", basis)` avoids reconstructing the basis from the file,
  but uses the passed basis instead. This save the time of constructing the basis
  twice and allows to specify parallelisation options (via the passed basis). Usually
  this is useful for continuing a calculation on a supercomputer or cluster.

See also the discussion on Input and output formats on JLD2 files.

## Checkpointing of SCF calculations
A related feature, which is very useful especially for longer calculations with DFTK
is automatic checkpointing, where the state of the SCF is periodically written to disk.
The advantage is that in case the calculation errors or gets aborted due
to overrunning the walltime limit one does not need to start from scratch,
but can continue the calculation from the last checkpoint.

The easiest way to enable checkpointing is to use the `kwargs_scf_checkpoints`
function, which does two things. (1) It sets up checkpointing using the
`ScfSaveCheckpoints` callback and (2) if a checkpoint file is detected,
the stored density is used to continue the calculation instead of the usual
atomic-orbital based guess. In practice this is done by modifying the keyword arguments
passed to # `self_consistent_field` appropriately, e.g. by using the density
or orbitals from the checkpoint file. For example:

In [5]:
checkpointargs = kwargs_scf_checkpoints(basis; ρ=guess_density(basis, magnetic_moments))
scfres = self_consistent_field(basis; tol=1e-2, checkpointargs...);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   α      Diag   Δtime
---   ---------------   ---------   ---------   ------   ----   ----   ------
  1   -27.64813879917                   -0.13    0.001   0.80    7.0    270ms
  2   -28.92457747594        0.11       -0.83    0.686   0.80    2.0    890ms
  3   -28.93109150637       -2.19       -1.13    1.189   0.80    2.0    149ms
  4   -28.93773303864       -2.18       -1.19    1.775   0.80    1.5   93.3ms
  5   -28.93929362905       -2.81       -1.09    2.000   0.80    1.5   93.6ms
  6   -28.93958184348       -3.54       -1.89    1.975   0.80    1.5   88.0ms
  7   -28.93961175330       -4.52       -2.94    1.985   0.80    1.0   86.5ms


Notice that the `ρ` argument is now passed to kwargs_scf_checkpoints instead.
If we run in the same folder the SCF again (here using a tighter tolerance),
the calculation just continues.

In [6]:
checkpointargs = kwargs_scf_checkpoints(basis; ρ=guess_density(basis, magnetic_moments))
scfres = self_consistent_field(basis; tol=1e-3, checkpointargs...);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   α      Diag   Δtime
---   ---------------   ---------   ---------   ------   ----   ----   ------
  1   -28.93961277456                   -3.32    1.985   0.80   10.0    197ms


Since only the density is stored in a checkpoint
(and not the Bloch waves), the first step needs a slightly elevated number
of diagonalizations. Notice, that reconstructing the `checkpointargs` in this second
call is important as the `checkpointargs` now contain different data,
such that the SCF continues from the checkpoint.
By default checkpoint is saved in the file `dftk_scf_checkpoint.jld2`, which can be changed
using the `filename` keyword argument of `kwargs_scf_checkpoints`. Note that the
file is not deleted by DFTK, so it is your responsibility to clean it up. Further note
that warnings or errors will arise if you try to use a checkpoint, which is incompatible
with your calculation.

We can also inspect the checkpoint file manually using the `load_scfres` function
and use it manually to continue the calculation:

In [7]:
oldstate = load_scfres("dftk_scf_checkpoint.jld2")
scfres   = self_consistent_field(oldstate.basis, ρ=oldstate.ρ, ψ=oldstate.ψ, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   Diag   Δtime
---   ---------------   ---------   ---------   ------   ----   ------
  1   -28.93839359681                   -2.27    1.985    5.5    126ms
  2   -28.93940969330       -2.99       -2.74    1.985    1.0   86.8ms
  3   -28.93961048575       -3.70       -2.54    1.985    4.5    124ms
  4   -28.93961182134       -5.87       -2.68    1.985    1.0   74.0ms
  5   -28.93961266009       -6.08       -2.86    1.985    1.0   74.1ms
  6   -28.93961281585       -6.81       -2.95    1.985    1.0   74.4ms
  7   -28.93961304902       -6.63       -3.18    1.985    1.0   75.2ms
  8   -28.93961310972       -7.22       -3.33    1.985    1.0   76.7ms
  9   -28.93961314821       -7.41       -3.57    1.985    1.5   80.8ms
 10   -28.93961316689       -7.73       -4.07    1.985    1.5   84.3ms


Some details on what happens under the hood in this mechanism: When using the
`kwargs_scf_checkpoints` function, the `ScfSaveCheckpoints` callback is employed
during the SCF, which causes the density to be stored to the JLD2 file in every iteration.
When reading the file, the `kwargs_scf_checkpoints` transparently patches away the `ψ`
and `ρ` keyword arguments and replaces them by the data obtained from the file.
For more details on using callbacks with DFTK's `self_consistent_field` function
see Monitoring self-consistent field calculations.

(Cleanup files generated by this notebook)

In [8]:
rm("dftk_scf_checkpoint.jld2")
rm("scfres.jld2")